In [ ]:
#!/usr/bin/python
# -*- coding: latin-1 -*-

import inspect, os, sys

try :
    import pywikibot as pb
    from pywikibot.specialbots import UploadRobot

except :
    current_folder = os.path.realpath(os.path.abspath(os.path.split(inspect.getfile(inspect.currentframe()))[0]))
    folder_parts = current_folder.split(os.sep)
    pywikibot_folder = os.sep.join(folder_parts[:-1])

    if current_folder not in sys.path:
        sys.path.insert(0, current_folder)
    if pywikibot_folder not in sys.path:
        sys.path.insert(0, pywikibot_folder)

    import pywikibot as pb
    from pywikibot.specialbots import UploadRobot

from bs4 import BeautifulSoup
from mako.template import Template
import requests
from requests.compat import quote

from PIL import Image
from io import BytesIO
import re

commons_site = pb.Site("commons", "commons")

In [ ]:
from modules.wmtools import is_commons_file, get_hash

In [ ]:
cwd = os.getcwd()

images_directory = os.path.join(cwd, 'images')
if not os.path.exists(images_directory):
    os.makedirs(images_directory)

In [ ]:
#### User input:
url = 'http://www.eldiario.es/politica/democracia-crisis-instituciones_0_234126756.html'

user_agent = 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) Gecko/20100101 Firefox/40.1'
headers = {'User-Agent' : user_agent}

#### User input:
categories = ['Ignacio Sánchez-Cuenca', '2014 in Spain']

#### User input:
author = ''

In [ ]:
#### User input:
upload_categories = ['Files uploaded by User:Discasto']
categories = categories + upload_categories
categories

In [ ]:
r = requests.get(url, headers=headers)
soup = BeautifulSoup(r.text, 'html.parser')

In [ ]:
# Publication date
pub_date = soup.find_all("span", class_="date")[0].get_text()
p = re.compile('[0-9]{2}\/[0-9]{2}\/[0-9]{4}')
m = p.match(pub_date)
pub_date = m.group(0).split('/')
pub_date.reverse()
pub_date='-'.join(pub_date)
pub_date

In [ ]:
title = soup.find_all("h1", class_="pg-headline")[0].get_text().strip().replace('  ', ' ')
#### User input:
#title = u""
title

In [ ]:
article_description = soup.find_all("div", class_="subtitle bkn mce sk2")
article_description = '. '.join([p.get_text() for p in article_description[0].find_all("p")]).replace('..', '.')
article_description

In [ ]:
image_list = []
for image in [image for image in soup.find_all("figure", class_="media-holder")]:
    item = {}
    image_url = image.find_all("img")[0]['src']
    p = re.compile('(.*_)[0-9]{1,2}\.jpg')
    m = p.match(image_url)
    item['image_url'] = '{}1.jpg'.format(m.group(1))

    image_title = image.find_all("img")[0]['alt'].split('.')[0].strip()
    image_title = image_title.split('/')[0].strip()
    item['image_title'] = image_title
    if image_title == 'plyr':
        continue

    if author :
        item['image_author'] = author
    else :
        try :
            image_author = image.find_all("span", class_="font")[0].get_text()
        except :
            if len (image.find_all("img")[0]['alt'].split('.')) > 1:
                image_author = image.find_all("img")[0]['alt'].split('.')[1].replace(' Foto: ', '')
            elif len (image.find_all("img")[0]['alt'].split('/')) > 1:
                image_author = image.find_all("img")[0]['alt'].split('/')[1]
            else :
                image_author = ''
        item['image_author'] = image_author.strip().title()

    image_list.append(item)
image_list

In [ ]:
template = u"""=={{int:filedesc}}==
{{Information
|description={{es|1=${article_description}}}
|date=${date}
|source=[${article_url} eldiario.es - ${article_title}]
|author=${author} ([http://www.eldiario.es/ eldiario.es])
|permission=[http://www.eldiario.es/licencia/ License]
|other versions=
}}

=={{int:license-header}}==
{{LicenseReview}}
{{cc-by-sa-3.0-es}}

${cat_string}"""

vars = {
    "article_url": url,
    "article_description": article_description,
    "date": pub_date,
    "article_title": title
}

In [ ]:
#### User input:
excluded = []

used_names = []

for i, image in enumerate(image_list):
    cats = categories[:]
    # If the image is excluded, skip
    if i in excluded:
        print ("Image excluded. Skipping")
        continue
        
    # First, the image is downloaded and stored
    image_url = quote(image["image_url"].encode('utf-8'), ':/')
    image_name = image["image_title"].replace(':', ' -').replace('  ', ' ') + '.jpg'
    print(image_name)
    image_path = os.path.join(images_directory, image_name)
    try: 
        r = requests.get(image_url, headers=headers)
        downloaded_image = Image.open(BytesIO(r.content))
        downloaded_image.save(image_path)
    except Exception as err:
        print ('Failed download ({}). Skipping'.format(err))
        continue

    # If the image is already in Commons, skip
    if is_commons_file(get_hash(image_path)) :
        print ("Image already in commons. Skipping")
        os.remove(image_path)
        continue

    # If the image name is already in commons, find a new name
    if pb.Page(commons_site, image_name, ns=6).exists():
        print ("Image name ({0}) already used in Commons".format(image_name))
        used_names.append(image_name)
        
    while True:
        if image_name in used_names :
            # Finding a new name
            image_subject = '.'.join(image_name.split('.')[:-1])
            image_extension = 'jpg'
            p = re.compile('(.*) ([0-9]{2}\.jpg)')
            m = p.match(image_name)
            if m is None:
                image_name = image_subject + ' 01.' + image_extension
            else :
                counter = int(m.group(2)[:2]) + 1
                image_name = '{} {:02d}.{}'.format(m.group(1), counter, image_extension)

            if pb.Page(commons_site, image_name, ns=6).exists():
                print ("Image name ({0}) already used in Commons. Finding a new name".format(image_name))
                used_names.append(image_name)
        else :
            print ("Preparing to upload image with name {0}".format(image_name))
            used_names.append(image_name)
            break

    # image upload
    if "Marta Jara" in image["image_author"]:
        vars["author"] = 'Marta Jara'
        cats.append('Photographs by Marta Jara (eldiario.es)')
    elif "Robert Bonet" in image["image_author"]:
        vars["author"] = '[http://www.eldiario.es/autores/robert_bonet/ Robert Bonet]'
        cats.append('Photographs by Robert Bonet (eldiario.es)')        
    elif "Sandra Lázaro" in image["image_author"]:
        vars["author"] = '[http://www.eldiario.es/autores/sandra_lazaro_-fotos/ Sandra Lázaro]'
        cats.append('Photographs by Sandra Lázaro (eldiario.es)')
    elif "Juan Manzanara" in image["image_author"] :
        vars["author"] = '[http://www.eldiario.es/autores/juan_manzanara/ Juan Manzanara]'
        cats.append('Photographs by Juan Manzanara (eldiario.es)')
    elif "Sònia Calvó" in image["image_author"]:
        vars["author"] = '[http://www.eldiario.es/autores/sonia_calvo/ Sònia Calvó]'
        cats.append('Files from eldiario.es')
    elif "Carlos Hernández" in image["image_author"]:
        vars["author"] = '[http://www.eldiario.es/autores/carlos_hernandez/ Carlos Hernández]'
        cats.append('Files from eldiario.es')
    elif "Enric Català" in image["image_author"]:
        vars["author"] = '[http://www.eldiario.es/autores/enric_catala_-fotos/ Enric Català]'
        cats.append('Files from eldiario.es')
    else :
        cats.append('Files from eldiario.es')
        
    vars["cat_string"] = '\n'.join(['[[Category:'+i+']]' for i in list(set(cats))])
    t = Template(template)
    _text = t.render(**vars)
    #print(_text)

    bot = UploadRobot([image_path],
                      description = _text,
                      useFilename = image_name,
                      keepFilename = True,
                      verifyDescription = False,
                      ignoreWarning = True,
                      targetSite = commons_site)
    bot.run()

    os.remove(image_path)